## Transformers: Semantic segmentation

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from transformers import (
    pipeline,
    AutoImageProcessor,
    AutoModelForSemanticSegmentation
)

## Load image

In [ ]:
# load image
url = 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/segmentation_input.jpg'
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
# show image
fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(np.asarray(image))
ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

## Import model

In [ ]:
# set model name
ckpt_name = 'nvidia/segformer-b1-finetuned-cityscapes-1024-1024'

In [ ]:
# create preprocessor
preprocessor = AutoImageProcessor.from_pretrained(ckpt_name)

print(preprocessor)

In [ ]:
# load model
model = AutoModelForSemanticSegmentation.from_pretrained(ckpt_name)
model = model.eval()

print(model)

In [ ]:
# load pipeline (preprocessor, model and postprocessor)
pipe = pipeline('image-segmentation', model=ckpt_name)

## Run model

In [ ]:
# preprocess image
preprocessed_images = preprocessor(image, return_tensors='pt')
x = preprocessed_images['pixel_values']

print(f'Input shape: {x.shape}')

In [ ]:
# run model
predictions = model(x)
y_logits = predictions.logits

print(f'Output shape: {y_logits.shape}')

In [ ]:
# get specific predictions
target_label = 'car'

target_idx = model.config.label2id[target_label]
target_logits = y_logits[0, target_idx].detach()

In [ ]:
# show predictions
fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(target_logits.sigmoid().numpy())
ax.set_aspect('equal', adjustable='box')
ax.set_title(f'predictions: {target_label}')
fig.tight_layout()

## Run pipeline

In [ ]:
# run pipeline
results = pipe(image)

print(results)

In [ ]:
# get specific predictions
pipe_labels = [d['label'] for d in results]

target_idx = pipe_labels.index(target_label)
target_mask = results[target_idx]['mask'] # PIL image

In [ ]:
# show predictions
fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(np.asarray(target_mask))
ax.set_aspect('equal', adjustable='box')
ax.set_title(f'predictions: {target_label}')
fig.tight_layout()